In [1]:
#Install and load library
# update.packages(ask = FALSE)
# install.packages("NLP", dependencies=TRUE)
# install.packages("slam", dependencies=TRUE)
# install.packages("tm", dependencies=TRUE) # for text mining
# install.package("SnowballC", dependencies=TRUE) # for text stemming
# install.packages("wordcloud", dependencies=TRUE)# word-cloud generator
# install.packages("RColorBrewer", dependencies=TRUE) # color palettes
# install.packages('lda', dependencies=TRUE)
# install.packages('modeltools', dependencies=TRUE)
# install.packages('stats4', dependencies=TRUE)
# # install.packages('methods', dependencies=TRUE)
# install.packages('toppicmodels', dependencies=TRUE)
# install.packages('ggplot2', dependencies = TRUE)
# install.packages("NbClust", dependencies = TRUE)
# install.packages("factoextra", dependencies = TRUE)

# install.packages("lda", dependencies = TRUE)
# install.packages("MASS", dependencies = TRUE)
# install.packages("topicmodels", dependencies = TRUE)
# install.packages("lsa", dependencies = TRUE)

In [2]:
setwd("/home/harish/PycharmProjects/Topic-Modeling/")

In [3]:
library(NLP)
library(tm)
library(SnowballC)
library(wordcloud)
library(RColorBrewer)
library(ggplot2)
library(factoextra)
# library(cluster)
library(NbClust)
library(lsa)
# library(fpc)

#LDA

library(lda)
library(MASS)
library(topicmodels)

Loading required package: RColorBrewer

Attaching package: ‘ggplot2’

The following object is masked from ‘package:NLP’:

    annotate

Welcome! Related Books: `Practical Guide To Cluster Analysis in R` at https://goo.gl/13EFCZ


In [4]:
## BUILDING CORPUS

folder <-"/home/harish/PycharmProjects/Topic-Modeling/Data Extraction/dataset/manual_93_2005"
summary(folder)
corpus_name <- Corpus(DirSource(folder, recursive=TRUE),readerControl = list(reader=readPlain));

   Length     Class      Mode 
        1 character character 

In [5]:
corpus_name

<<SimpleCorpus>>
Metadata:  corpus specific: 1, document level (indexed): 0
Content:  documents: 102

In [6]:
#---------------Data preprocessing:
print("dimension before:");
print(DocumentTermMatrix(corpus_name));

corpus_name<-tm_map(corpus_name,PlainTextDocument);
corpus_name<-tm_map(corpus_name, content_transformer(tolower));
corpus_name<-tm_map(corpus_name,removeWords,stopwords("english"));
corpus_name<-tm_map(corpus_name,removePunctuation);
corpus_name<-tm_map(corpus_name,removeNumbers);
corpus_name<-tm_map(corpus_name,stripWhitespace);

words_to_remove_in_article<-c("system","reserve","tthe","rnthe", "continue", "open","committee", "federal", "also", "meeting", "FOMC", "\r","\t","Present", "\n", 'year') #irrevalant words
corpus_name<-tm_map(corpus_name, removeWords,words_to_remove_in_article); #removing irrevalant words in the article

corpus_name<-tm_map(corpus_name, stemDocument, language="english");

print("dimension after:");
print(DocumentTermMatrix(corpus_name));

[1] "dimension before:"
<<DocumentTermMatrix (documents: 102, terms: 7507)>>
Non-/sparse entries: 100833/664881
Sparsity           : 87%
Maximal term length: 20
Weighting          : term frequency (tf)
[1] "dimension after:"
<<DocumentTermMatrix (documents: 102, terms: 10198)>>
Non-/sparse entries: 83679/956517
Sparsity           : 92%
Maximal term length: 142
Weighting          : term frequency (tf)


In [7]:
#todo: change the bounds, weighttfidf, 

In [8]:
#----------------------Text Analysis------

##build tdm/dtm matrix
tdm <- TermDocumentMatrix(corpus_name,control=list(wordLengths=c(4,Inf),bounds = list(global = c(2,Inf))))
tdm_matrix <- as.matrix(tdm)
print(dim(tdm_matrix))
##build a document/term matrix... words must have length 4
dtm <- DocumentTermMatrix(corpus_name,control=list(wordLengths=c(4,Inf),bounds = list(global = c(2,Inf))))
dtm_matrix <- as.matrix(dtm)
print(dim(dtm_matrix))

[1] 4091  102
[1]  102 4091


In [9]:
dtms <- removeSparseTerms(dtm, 0.93)

In [10]:
m <- as.matrix(tdm)
v <- sort(rowSums(m), decreasing=TRUE)
d <- data.frame(word = names(v),freq=v)
print(head(d, 10))
# print(findFreqTerms(dtm,lowfreq = 10))

           word freq
growth   growth 2970
price     price 2911
econom   econom 2238
increas increas 2210
inflat   inflat 1934
polici   polici 1915
quarter quarter 1895
busi       busi 1494
consum   consum 1450
period   period 1438


## LDA:

In [11]:
# svs = sort.list(abs(svd(dtms)$v[,4]), decreasing = TRUE)
# dtms$dimnames$Terms[head(svs, 6)]

In [12]:

##return top words by concept

##first create function to return top words; require dtms
concept<-function(num){ 
  sv<-sort.list((svd(dtms))$v[,num],decreasing = FALSE)
  # print(sv)
  # print(dtms$dimnames)
  dm<-dtms$dimnames$Terms[head(sv,5)] 
  return(dm)
}

In [13]:
##how many words?
num <- 102
i <- 1:num
val <- sapply(i, concept)

In [14]:
# write.table(val,file=paste("/home/harish/PycharmProjects/Topic-Modeling/val.csv"), append = T)
# ldaGibbs5@gamma

In [15]:
# ##how many words?
# num <- 5
# i <- 1:num
# sapply(i, concept)
# # https://stackoverflow.com/questions/14875493/lda-with-topicmodels-how-can-i-see-which-topics-different-documents-belong-to


In [16]:
# k=5
# ldaGibbs5 <-LDA(dtms, k, method = "Gibbs") 
# #docs to topics 
# ldaGibbs5.topics <- as.matrix(topics(ldaGibbs5))
# #get probability of each topic in each doc
# topicProbabilities <- as.data.frame(ldaGibbs5@gamma)

In [17]:
# topicProbabilities
# nrow(ldaGibbs5@gamma)

In [18]:
# vector <- NULL
# for(i in 1:nrow(ldaGibbs5@gamma)) {
#   vector <- c(vector, ldaGibbs5@gamma[i,])
# }


# Concepts <- rep(c("Concept 1","Concept 2","Concept 3", "Concept 4","Concept 5"),times=102)#why 80?
# TimeByDocs <- as.numeric(rep(1:102,each=5))
# chartdata <- data.frame(Concepts,TimeByDocs,vector)
# myplot <- ggplot(chartdata, aes(x=TimeByDocs,y=vector,fill=Concepts)) + geom_area()


In [19]:
# ggsave(filename="myPlot.png", plot=myplot)

In [20]:
# l<-c()
# i <- 0
# for (j in 1993:2005){
#    l[i] <- j
#     i <- i + 1
# }

In [21]:
##how many words?
# for (k in 2:5){

    num <- 102
    i <- 1:num
    val <- sapply(i, concept)
    # https://stackoverflow.com/questions/14875493/lda-with-topicmodels-how-can-i-see-which-topics-different-documents-belong-to
    
k = 4

    ldaGibbs_k <-LDA(dtms, k, method = "Gibbs") 
    #docs to topics 
    ldaGibbs_k.topics <- as.matrix(topics(ldaGibbs_k))
    #get probability of each topic in each doc
    topicProbabilities <- as.data.frame(ldaGibbs_k@gamma)
write.table(val,file=paste("/home/harish/PycharmProjects/Topic-Modeling/val.csv"), append = T)

    vector <- NULL
    for(i in 1:nrow(ldaGibbs_k@gamma)) {
      vector <- c(vector, ldaGibbs_k@gamma[i,])
    }


    Concepts <- rep(c("Concept 1","Concept 2","Concept 3", "Concept 4","Concept 5"),times=102)#why 80?
    TimeByDocs <- as.numeric(rep(1:102,each=k))
    chartdata <- data.frame(Concepts,TimeByDocs,vector)
    myplot <- ggplot(chartdata, aes(x=TimeByDocs,y=vector,fill=Concepts)) + geom_area()
    plotname <- paste(k,"png", sep=".");
    ggsave(filename=plotname, plot=myplot)
    
# }

Warning message in write.table(val, file = paste("/home/harish/PycharmProjects/Topic-Modeling/val.csv"), :
“appending column names to file”Saving 6.67 x 6.67 in image


In [22]:
# Concepts <- rep(c("Concept 1","Concept 2","Concept 3", "Concept 4","Concept 5"),times=102)#why 80?
#     TimeByDocs <- as.numeric(rep(c("1993","1994","1995","1996","1997","1998","1999",
#                                    "2000","2001","2002","2003","2004","2005"),each=k))
#     chartdata <- data.frame(Concepts,TimeByDocs,vector)
#     myplot <- ggplot(chartdata, aes(x=TimeByDocs,y=vector,fill=Concepts)) + geom_area()
# myplot

In [23]:

write.table(topicProbabilities,file=paste("/home/harish/PycharmProjects/Topic-Modeling/val.csv"), append = T)

Warning message in write.table(topicProbabilities, file = paste("/home/harish/PycharmProjects/Topic-Modeling/val.csv"), :
“appending column names to file”